This is a notebook to execute and test the different miscellaneous functions implemented in `misc/arrays.py`. 

## `shift_array`

In [1]:
import numpy as np
def shift_array(t0, y0, shift, nmax=10, tab=0):
    """     
    shift the array and interpolate when the shift is smaller than the time step


    Parameters    
    ----------

    t0: array-like
        x

    y0: array-like
        y

    shift: float
        value to shift x and y    

    nmax : int
        maximum interpolation step

    tab: int
        unless said otherwise (tab != 0), replace values with 0. Default 0 


    Returns
    -------

    t, y: array-like
        shifted arrays


    Note
    ----

    Only tested with constant timesteps    
    """

    if shift == 0:
        return t0, y0

    t = np.copy(t0)
    y = np.copy(y0)

    # Get the interpolation step required
    dt = t[1] - t[0]
    n = dt / (abs(shift) % dt)
    n = round(n)

    if n > nmax:
        print("Interpolation step required", n, "has been reduced to", nmax)
        n = nmax

    if n > 1:
        # Interpolate
        step = (max(t) - min(t)) / (len(t) - 1) / n
        tint = np.arange(min(t), max(t), step)
        tck = interpolate.splrep(t, y, s=0)
        y = interpolate.splev(tint, tck, der=0)
        t = tint

    # Find shift coordinates (don't forget the negative shift case)
    if shift > 0:
        try:
            n = np.argmax(t >= (t[0] + shift))
        except ValueError:  # Empty sequence (shift probably too important)
            n = len(t)
    else:
        try:
            n = np.argmax(t >= (t[-1] + shift))
        # Empty sequence (negative shift probably too important)
        except ValueError:
            n = 0

    # Shift
    if shift > 0:
        y[n:] = y[:-n]
        y[:n] *= 0
        y[:n] += tab
    else:
        y[:n] = y[-n:]
        y[n:] *= 0
        y[n:] += tab

    return t, y

In [2]:
x = [1,2,3,4]
y = [10,15,20,25]
shift_array(x,y,2)

Interpolation step required inf has been reduced to 10


/home/pankaj/anaconda3/envs/radis-dev/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in long_scalars


NameError: name 'interpolate' is not defined

## `find_nearest`

In [49]:
def find_nearest(array, searched, returnarg=False):
    """ Return the closest elements in array of 'searched' array. 
    Also returns a boolean index
    
    Examples
    --------
    
    ::
        
        from numpy import array
        find_nearest(array([1,2,3,4], array([2.1,2])
        
        >>> (array([2]), array([False, True, False, False], dtype=bool))
        
        find_nearest(np.array([1,2,3,4]), np.array([2.6,2]))
        
        >>> (array([2, 3]), array([False,  True,  True, False], dtype=bool))

    """

    b = np.zeros_like(array, dtype=bool)

    #    def find_nearest(array,value):
    #    '''  assuming array is sorted. '''
    #        idx = np.searchsorted(array, value, side="left")
    #        print('idx',idx)
    #        if math.fabs(value - array[idx-1]) < math.fabs(value - array[idx]):
    #            return idx-1
    #        else:
    #            return idx

    def find_nearest(array, value):
        return (np.abs(array - value)).argmin()

    try:
        for s in searched:
            b[find_nearest(array, s)] = True
    except:
        b[find_nearest(array, searched)] = True

    if returnarg:
        out = array[b], b, np.argmax(b)
    else:
        out = array[b], b

    return out

In [50]:
from numpy import array
find_nearest(array([1,3,5,7,9,10,13,20]),array([5,12,11.5]))

(array([ 5, 10, 13]),
 array([False, False,  True, False, False,  True,  True, False]))

`find_nearest`works fine even with an unsorted array provided `searched` array has a single variable

In [120]:
find_nearest(array([50, 17, 8.9, 30, 5, 2, 1, 100, 25]), array([24]))

(array([25.]),
 array([False, False, False, False, False, False, False, False,  True]))

However, on passing multiple elements in `searched`, the return value is haphazard and the output is a sorted array, not in sync with the input `searched array`

In [122]:
find_nearest(array([50, 17, 8.9, 30, 5, 2, 1, 100, 25]), array([24, 10]))

(array([ 8.9, 25. ]),
 array([False, False,  True, False, False, False, False, False,  True]))

In [123]:
find_nearest(array([50, 17, 8.9, 30, 5, 2, 1, 100, 25]), array([10, 24]))

(array([ 8.9, 25. ]),
 array([False, False,  True, False, False, False, False, False,  True]))

In case the answer is same for multiple elements of `searched` array, it appears only once in the output

In [125]:
find_nearest(array([1, 2, 5, 8.9,17,25,30,50, 100]), array([24,26]))

(array([25.]),
 array([False, False, False, False, False,  True, False, False, False]))

In [127]:
find_nearest(array([1, 2, 5, 8.9,17,25,30,50, 100]), array([24,10, 26, 8, 0.5]))

(array([ 1. ,  8.9, 25. ]),
 array([ True, False, False,  True, False,  True, False, False, False]))

## `is_sorted` and `is_sorted_backward`

In [13]:
def is_sorted_backward(a):
    """ Returns whether ``a`` is sorted in descending order
    
    See Also
    --------
    
    :func:`~radis.misc.arrays.is_sorted`
    """
    for i in range(a.size - 1):
        if a[i + 1] > a[i]:
            return False
    return True

In [19]:
is_sorted_backward(array([111,100,13,1]))

True

In [20]:
def is_sorted(a):
    """ Returns whether ``a`` is sorted in ascending order
    
    From B.M. answer on StackOverflow: https://stackoverflow.com/a/47004533/5622825
    
    See Also
    --------
    
    :func:`~radis.misc.arrays.is_sorted_backward`
    """
    for i in range(a.size - 1):
        if a[i + 1] < a[i]:
            return False
    return True

In [22]:
is_sorted(array([10,2,3]))

False






## `bining`

In [23]:
def bining(I, ymin=None, ymax=None, axis=1):
    """ Averages a I multi-dimensional array (typically an image) along the y axis
    bining(I) corresponds to I.mean(axis=1)
    Nan are not taken into account
    Parameters    
    ----------
    I: numpy array
        intensity
    ymin: int [0-I.shape[1]]
        If None, 0 is used. Default ``None``.
    ymax: int [0-I.shape[1]]
        If None, I.shape[1] is used. Default ``None``.
    axis: int
        Default 1
    """
    I = np.array(I)  # convert to array in case it's a Pandas dataframe for instance
    if ymin is None:
        ymin = 0
    if ymax is None:
        ymax = I.shape[axis]
    if ymin < 0:
        print("Warning in bining. ymin ({0}) < 0".format(ymin))
    if ymax > I.shape[axis]:
        print(
            "Warning in bining. ymax ({0}) > yaxis length ({1})".format(
                ymax, I.shape[axis]
            )
        )
    return np.nanmean(I[:, ymin:ymax], axis=axis)

In [30]:
x = np.array([[10, 2, 31], [5, 8, 12], [11,11,11], [0,-1,4]])
print(x)

[[10  2 31]
 [ 5  8 12]
 [11 11 11]
 [ 0 -1  4]]


In [32]:
print(bining(x))

[14.33333333  8.33333333 11.          1.        ]


Final test for bining

In [89]:
a = np.arange(20).reshape(4,5)
print(a)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


In [119]:
b = bining(a)
print((b))
b == array([2, 7, 12, 17])
c = bining(a, ymin= 1)
print(c)
assert (c==array([2.5, 7.5, 12.5, 17.5])).all()
d = bining(a, ymax= 3)
print(d)
e = bining(a, ymin =1, ymax = )
print(e)

[ 2.  7. 12. 17.]
[ 2.5  7.5 12.5 17.5]
[ 1.  6. 11. 16.]
Warning in bining. ymax (6) > yaxis length (5)
[ 2.5  7.5 12.5 17.5]


In [114]:
assert 1==1

AssertionError: 

# `find_first`

In [33]:
def find_first(arr, treshold):
    """ Return the index of the first element of the array arr whose value
    is more than the treshold """
    return np.argmax(arr > treshold)

`find_first` returns `0` when no element in the array exceeds the threshold

In [71]:
print(find_first(array([5,1,8,10,13,2,20]), 9))

3


In [76]:
print(find_first(array([5,1,8,10,13,2,20]), 28))

0


In [74]:
print(find_first(array([5,1,8,10,13,2,20]), 11))

4


Final test for `find_first`

In [87]:
a = np.arange(10)
print(a)
print(find_first(a, -1) == 0)
print(find_first(a, 0) == 1)
print(find_first(a, 5) == 6)
print(find_first(a, 8) == 9)
print(find_first(a, 9) == 0)
print(find_first(a, 20) == 0)

[0 1 2 3 4 5 6 7 8 9]
True
True
True
True
True
True


In [63]:
a = np.arange(10)

a[::-1]
#print(is_sorted(a))
#is_sorted_backward(a[::-1])

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

## `calc_diff`

In [164]:
from scipy import interpolate
def calc_diff(t1, v1, t2, v2):
    """ Substract two vectors that may have slightly offset abscisses 
    interpolating the correct values 
    Parameters    
    ----------
    t1, v1: array_like
        first vector and its abscisses
    t2, v2: array_like
        second vector and its abscisses
    Returns
    -------
    tdiff, vdiff: array_like
        substracted vector and its abscisses
    """

    t1, v1, t2, v2 = list(map(np.array, (t1, v1, t2, v2)))

    # Deal with inversely sorted case
    if t1[-1] < t1[0]:
        t1, v1 = t1[::-1], v1[::-1]
    if t2[-1] < t2[0]:
        t2, v2 = t2[::-1], v2[::-1]

    # Get the overlapping range
    b = np.logical_and(t2 > t1[0], t2 < t1[-1])

    tdiff = t2[b]
    v2 = v2[b]

    # Interpolate the correct values
    f = interpolate.interp1d(t1, v1)
    v1 = f(tdiff)

    # Finally, substract:
    vdiff = v1 - v2

    return tdiff, vdiff

In [138]:
t1 = array([1,2,3,4,5,6])
v1 = array([4.5, 2.0, 3.0, 3.5, 4.0, 5.0])

t2 = array([3,4,5,6,7,8])
v2  = array([1.5, 5.0, 1.0, 2.5, 3.5, 4.0])

td, vd = calc_diff(t1, v1, t2, v2)

In [139]:
print(td)
print(vd)

[3 4 5]
[ 1.5 -1.5  3. ]


In [145]:
x1 = array([1,2,3])
x2 = array([1,2,3])

In [182]:
#t1 = np.arange(5)
#t2 = np.arange(5)
t1 = array([1,2,3,4,5])
t2 = array([1,2,3,4,5])
v1 = np.array([10,12,14,16,18])
v2 = np.array([10,12,14,16,18])

tres, vres = calc_diff(t2[::-1], v1, t2, v2)
# print(type(tres))
# print(type(t1))
# print(type(t2))
print(tres)
print(vres)
# print(t1)
# print(t2)
#assert (calc_diff(t1, v1, t2, v2) == array([2,3,4]), array([0,0,0])
#assert (tres == array([2,3,4])).all()
#assert (vres == array([0,0,0])).all()

[2 3 4]
[ 4.  0. -4.]


In [168]:
t1+2

array([3, 4, 5, 6, 7])